In [78]:
import ast
import pandas as pd
df = pd.read_csv('data/split_embedqa.csv')
df.head()

,Context,Questions,Answers,embedding,Embedding
0,Berdasarkan barang bukti berikut: 2 (dua) pake...,1. Apa barang bukti yang ditemukan dalam kasus...,1. Barang bukti yang ditemukan dalam kasus tin...,"[0.02253102, -0.049208853, -0.025729991, 0.027...","[0.02253102, -0.049208853, -0.025729991, 0.027..."
1,Berdasarkan barang bukti berikut: 2 (dua) pake...,2. Apa jenis tindak pidana yang terbukti dilak...,2. Jenis tindak pidana yang terbukti dilakukan...,"[-0.04201769, 0.00092172425, -0.03313809, 0.03...","[-0.041427433, 0.0008820686, -0.03380097, 0.03..."
2,Berdasarkan barang bukti berikut: 2 (dua) pake...,3. Berapa lama pidana penjara yang dijatuhi ke...,3. Pidana penjara yang dijatuhi kepada Terdakw...,"[-0.017504673, 0.0043885163, 0.04658693, 0.031...","[-0.017504673, 0.0043885163, 0.04658693, 0.031..."
3,Berdasarkan barang bukti berikut: 2 (dua) pake...,4. Apa yang akan terjadi jika Terdakwa tidak d...,4. Jika Terdakwa tidak dapat membayar pidana d...,"[-0.04546008, 0.022565776, 0.020183947, 0.0533...","[-0.04546008, 0.022565776, 0.020183947, 0.0533..."
4,Berdasarkan barang bukti berikut: 2 (dua) pake...,5. Apa keputusan hakim mengenai status barang ...,5. Keputusan hakim mengenai status barang bukt...,"[0.014472564, -0.018997155, -0.050127976, 0.03...","[0.014472564, -0.018997155, -0.050127976, 0.03..."


In [79]:
from openai import OpenAI
import os

In [80]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [81]:
client = OpenAI()

In [82]:
def embedding_query(query):
    try:
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=query,
            encoding_format="float"
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"An error occurred: {e}")

In [86]:
from scipy.spatial.distance import cosine
import pandas as pd

def rank_distances(query_embedding, embeddings_list):
    distances = [(i, cosine(query_embedding, ast.literal_eval(embedding))) for i, embedding in enumerate(embeddings_list)]
    return sorted(distances, key=lambda x: x[1])

query = "Apa saja barang bukti yang ditemukan dan dijadikan dasar dalam putusan narkotika?"
query_embedding = embedding_query(query)
ranked_results = rank_distances(query_embedding, df["Embedding"])

# Retrieve the top 5 results from ranked_results with respective ranks and answers
top_5_results = [
    {"No": idx + 1, "Rank": rank, "Answer": df["Answers"][i]}
    for idx, (i, rank) in enumerate(ranked_results[:5])
]

# Create a DataFrame for display
top_5_df = pd.DataFrame(top_5_results)
top_5_df

,No,Rank,Answer
0,1,0.277329,1. Jenis barang bukti yang disebutkan dalam ke...
1,2,0.283003,1. Jenis narkotika yang ditemukan sebagai bara...
2,3,0.283558,1. Jenis narkotika yang ditemukan sebagai bara...
3,4,0.289102,3. Barang bukti yang dirampas untuk dimusnahka...
4,5,0.294043,3. Barang bukti lain yang disita selain narkot...
